<a href="https://colab.research.google.com/github/swilsonmfc/linear/blob/main/Camping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Camping

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjgxE2QpU7Bxqf5qLn7oaegUij4VjXy3tmew&usqp=CAU" width="400"/>

# Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data

In [2]:
camping_df = pd.read_csv('https://raw.githubusercontent.com/swilsonmfc/linear/main/camping.csv')

In [3]:
camping_df

,LIVE_BAIT,CAMPER,PERSONS,CHILDREN,FISH_COUNT
0,0,0,1,0,0
1,1,1,1,0,0
2,1,0,1,0,0
3,1,1,2,1,0
4,1,0,1,0,1
...,...,...,...,...,...
245,1,1,2,0,0
246,1,1,4,3,0
247,1,1,2,1,0
248,1,1,3,2,0


# Modeling Task
* Predict the count of fish caught for a camping party